[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jieenny/GoogleEarthEngine-for-Remote-sensing/blob/main/Monitoring_Iceberg_Time_Series_Area_Data_Tutorial.ipynb)





---------------------------




## **Monitoring Iceberg Time Series Area Data Tutorial** 
## **빙하 면적 변화 시계열 그래프 분석 튜토리얼** 


----------------------------



> In this tutorial I used iceberg 'A77'


> This tutorial is divided into three parts
 * **Part1:** A77 Iceberg area detection   (original link: https://colab.research.google.com/drive/1fcTuR9vMlq6WMHX_oABNqV-8X4MpHBOh)
 * **Part2:** A77 Iceberg Time Series Area Data Exporting and Pre-processing
 * **Part3:** Monitoring Time Series Area Graph
 











# **Part1: A77 iceberg area detection**



#### 1) 필요한 패키지 설치




In [ ]:
# Google Earth Engine (GEE) 패키지 가져오기
import ee

ee.Authenticate()
ee.Initialize()

In [ ]:
# 필요한 python 패키지 가져오기
import pandas as pd
import altair as alt

#### 2) 원하는 ImageCollection 및 roi 가져온 후 이미지 시각화

In [ ]:
# shapefile (.shp)을 이용하여 roi (region of interest) 설정하기
shp = ee.FeatureCollection('projects/ee-jieun/assets/A77') # check
coor = shp.geometry().getInfo()['coordinates']
roi = ee.Geometry.Polygon(coor)

# 날짜 설정하기
start = ee.Date('2021-08-01')
end = ee.Date('2021-08-08') 

# Sentinel-1 SAR 이미지 가져오기
image = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .select('HH')
    .mean()
    .clip(roi))

In [ ]:
# Sentinel-1 SAR 이미지 시각화하기 
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image titles to folium ma
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Mapbox attribution',
        name = name,
        overlay = True,
        control = True 
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 8)

# Add the layer to the map object.
map.add_ee_layer(image, {'min': [-22], 'max': [2]}, 'HH')

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)

#### 3) 빙하와 해양 구별한 후 빙하 면적 구하기

In [ ]:
# HH 값의 히스토그램 그리기
reduce_image = image.clipToBoundsAndScale(geometry=roi, scale=70)
histogram = reduce_image.reduceRegion(ee.Reducer.histogram(), roi)
x = ee.Array(ee.Dictionary(histogram.get('HH')).get('bucketMeans')).getInfo()
y = ee.Array(ee.Dictionary(histogram.get('HH')).get('histogram')).getInfo()

xy = pd.DataFrame({'HH': x, 'Number of pixels': y})
alt.Chart(xy).mark_bar().encode(x='HH', y='Number of pixels')

In [ ]:
# 빙하 (glacier)와 해양을 구별한 후 시각화하기
disc = -7     # 빙하 구분 기준값
image_mask = image.gt(disc) 

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image titles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Mapbox attribution',
        name = name,
        overlay = True,
        control = True 
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 9)

# Add the layer to the map object.
map.add_ee_layer(image_mask, {'min': [0], 'max': [1]}, 'HH')

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)

In [ ]:
# 빙하 (glacier) 면적 계산하기
Area = ee.Image.pixelArea()
pixelArea = image_mask.multiply(Area).rename('Area')
sumArea = int(round(pixelArea.reduceRegion(reducer=ee.Reducer.sum(), geometry=roi, scale=200).get('Area').getInfo())/10**6) # m^2 -> km^2

sumArea

# **Part2: A77 iceberg Time Series area data exporting and pre-processing**

#### 1) 원하는 roi의 ImageCollection 불러오기



> 온전한 빙하 면적을 담고 있는 이미지의 pass number로 필터링    
ASF data의 S1_GRD pass number에 해당하는 property: 
 * *relativeOrbitNumber_start*:	(DOUBLE)	Relative orbit number of the oldest line within the image data.


In [ ]:
# roi (region of interest) 설정하기
shp = ee.FeatureCollection('projects/ee-jieun/assets/A77') # check
coor = shp.geometry().getInfo()['coordinates']
roi = ee.Geometry.Polygon(coor)

# 날짜 설정하기
start = ee.Date('2021-01-01')
end = ee.Date('2022-08-01') 

# Sentinel-1 SAR 이미지 가져오기
Images = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .filter(ee.Filter.eq('relativeOrbitNumber_stop', 75))             # A77 iceberg의 경우
    .select('HH')
)

#### 2) Reducing 함수 구현 후 Data Export
time series 각 이미지에서 빙하 면적 계산

In [ ]:
# 이미지에 마스크 씌워 새로운 이미지 pixelArea 반환하는 함수 작성
disc = -7   # 빙하 구분 기준값 설정
def to_pixelArea(image):
  image_mask = image.gt(disc)
  Area = ee.Image.pixelArea()
  pixelArea = image_mask.multiply(Area).rename('Area')
  return pixelArea.set({'system:time_start': image.get('system:time_start')})

In [ ]:
# Image에 mapping할 reduce 함수 작성
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.sum(),
                                  scale=200,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  def reduce_region_function(pixelArea):  
    stat = pixelArea.select('Area').reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': pixelArea.date().millis()}) 
  return reduce_region_function

In [ ]:
# reducer 객체 생성
reduce_Images = create_reduce_region_function(
    geometry=roi, reducer=ee.Reducer.sum(), scale=200, crs='EPSG:4326')

In [ ]:
# mask 씌운 imageCollection 생성
Images_mask = Images.map(to_pixelArea)
type(Images_mask)

In [ ]:
# mask 씌운 이미지를 reduce하여 빙하 면적을 계산한 FeatureCollection 생성
stat_fc = ee.FeatureCollection(Images_mask.map(reduce_Images))

In [ ]:
# Export 함
task = ee.batch.Export.table.toAsset(
    collection=stat_fc,
    description='stat_fc export',
    #assetId='projects/ee-jieun/assets/stat_fc-6')     # check and run

task.start()

In [ ]:
# 진행상황 확인
task.status()   # completed: 완료

#### 3) Data 로드 후 dataframe 생성

In [ ]:
# 로드
#stat_fc = ee.FeatureCollection('projects/ee-jieun/assets/stat_fc-6')     # check and run

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

In [ ]:
# feature properties를 dictionary로 변환
images_dict = fc_to_dict(stat_fc).getInfo()

In [ ]:
# dictionary 정보 확인
print(type(images_dict), '\n')
for prop in images_dict.keys():
    print(prop + ':', images_dict[prop][0:3] + ['...'])

In [ ]:
# dataframe으로 변환; images_df
images_df = pd.DataFrame(images_dict)

In [ ]:
# 'Area' 면적 단위 변환 m^2 -> km^2
import numpy as np
images_df['Area'] = (np.array(images_dict['Area']) / 10**6).astype(int)
display(images_df)
print(images_df.dtypes)

In [ ]:
# DataFrame에 날짜 변수 생성
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

images_df = add_date_info(images_df)
images_df.head(10)




*   12일 간격으로 데이터 있음



In [ ]:
# 데이터 정제; images_df_cln에 저장
images_df_cln = images_df.drop(['millis', 'system:index'], axis=1)
images_df_cln.sort_values(['Year','DOY'], inplace=True)
images_df_cln.reset_index(inplace=True)
images_df_cln.columns = ['index', 'sumArea', 'Timestamp', 'Year', 'Month', 'Day', 'DOY']  #index는 원데이터를 불러올 때 사용할 것이므로 남겨둔다
# 확인
images_df_cln.head(10)

In [ ]:
images_df_cln.tail(10)



*   시간 순으로 정렬하면 2021년은 6일 간격으로, 2022년은 12일 간격으로 데이터 존재

*   *images_df_cln*: 시계열 분석을 위한 최종 데이터셋 완성



---







# **Part3: Monitoring Time Series area graph**



#### 1) 시간에 따른 빙하 면적 그래프 그리기


In [ ]:
# line graph
iceberg_name = 'A77'
ts = pd.DataFrame({'date': images_df_cln['Timestamp'], 'sumArea': images_df_cln['sumArea']})
alt.Chart(ts, width=1000).mark_line().encode(x='date', y='sumArea').properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프')

In [ ]:
# area graph
iceberg_name = 'A77'
ts = pd.DataFrame({'date': images_df_cln['Timestamp'], 'sumArea': images_df_cln['sumArea']})
alt.Chart(ts, width=1000).mark_area().encode(x='date', y='sumArea', color='Day').properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프')


*그래프 해석*
*   2021년 8월까지 일정치를 유지하다가 8월 중순 경에 급격한 감소를 보인다
*   전체적인 추세를 살펴보면 2021년 8월 중순 경, 2022년 1월 중순 경 큰 변화가 보인다
*   큰 변화를 나타낸 시점의 데이터만 세부 분석을 진행한다 (2021-08월, 2021-11월, 2022-01월)



#### 2) 전체 시계열 데이터에서 변화가 나타난 시점 세부 분석



> 2021년 8월 데이터



In [ ]:
iceberg_name = 'A77'
month = 8; year=2021  # check
cond =  (images_df_cln['Month']==month) & (images_df_cln['Year']==year)
# 조건(cond)에 해당하는 데이터만 추출
ts = pd.DataFrame({'date': images_df_cln[cond]['Timestamp'], 'sumArea': images_df_cln['sumArea']})
alt.Chart(ts,width=500).mark_line().encode(x=alt.X('date'), y=alt.Y('sumArea')).properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프 '+str(year)+'년 '+str(month)+'월 ')



*   2021년 8월 11일에서 17일 사이에 빙하 면적 감소함





> 2021년 11월 데이터



In [ ]:
iceberg_name = 'A77'
month = 11; year=2021  # check
cond =  (images_df_cln['Month']==month) & (images_df_cln['Year']==year)
# 조건(cond)에 해당하는 데이터만 추출
ts = pd.DataFrame({'date': images_df_cln[cond]['Timestamp'], 'sumArea': images_df_cln['sumArea']})
alt.Chart(ts,width=500).mark_line().encode(x=alt.X('date'), y=alt.Y('sumArea')).properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프 '+str(year)+'년 '+str(month)+'월 ')



*   2021년 11월 15일 이후 27일까지 빙하 면적 증가





> 2022년 1월 데이터



In [ ]:
iceberg_name = 'A77'
month = 1; year=2022  # check
cond =  (images_df_cln['Month']==month) & (images_df_cln['Year']==year)
# 조건(cond)에 해당하는 데이터만 추출
ts = pd.DataFrame({'date': images_df_cln[cond]['Timestamp'], 'sumArea': images_df_cln['sumArea']})
alt.Chart(ts,width=500).mark_line().encode(x=alt.X('date'), y=alt.Y('sumArea')).properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프 '+str(year)+'년 '+str(month)+'월 ')



*   2022년 1월 14일 쯤에 면적이 거의 0에 수렴하게 감소


#### 3) 해당 시점의 Sentinel-1 원본 이미지 시각화하여 확인




##### 2021년 8월 16일
>  빙하 면적 감소 현상


In [ ]:
# 2021년 8월 16일 즈음의 Sentinel-1 이미지
start = ee.Date('2021-08-16')
end = ee.Date('2021-08-17') 

# Sentinel-1 SAR 이미지 가져오기
image0817 = (ee.ImageCollection('COPERNICUS/S1_GRD')      # changing your image variable name
    .filterDate(start, end)
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .select('HH')
    .mean()
    .clip(roi))

# shapefile (.shp)을 이용하여 roi (region of interest) 설정하기
shp = ee.FeatureCollection('projects/ee-jieun/assets/A77') # check
coor = shp.geometry().getInfo()['coordinates']
roi = ee.Geometry.Polygon(coor)

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 8)

# Add the layer to the map object.
map.add_ee_layer(image0817, {'min': [-22], 'max': [2]}, 'HH')    # changing your image variable name

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)

In [ ]:
# 빙하 (glacier)와 해양을 구별한 후 시각화하기
disc = -7    # 기준값
image_mask = image0817.gt(disc)       #changing your image variable name

# Import the Folium library.
import folium

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 9)

# Add the layer to the map object.
map.add_ee_layer(image_mask, {'min': [0], 'max': [1]}, 'HH')

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)



*   분석 결과, 빙하가 실제로 떨어져 나간 것을 확인할 수 있다





##### 2021년 11월 27일 
> 빙하 면적 증가 현상



In [ ]:
# 2021년 11월 27일의 Sentinel-1 이미지 
start = ee.Date('2021-11-26')
end = ee.Date('2021-11-27')

# Sentinel-1 SAR 이미지 가져오기
image1127 = (ee.ImageCollection('COPERNICUS/S1_GRD')     # changing your image variable name
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .select('HH')
    .mean()
    .clip(roi))

# shapefile (.shp)을 이용하여 roi (region of interest) 설정하기
shp = ee.FeatureCollection('projects/ee-jieun/assets/A77') # check
coor = shp.geometry().getInfo()['coordinates']
roi = ee.Geometry.Polygon(coor)

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 8)

# Add the layer to the map object.
map.add_ee_layer(image1127, {'min': [-22], 'max': [2]}, 'HH')    # changing your image variable name

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)

In [ ]:
# 빙하 (glacier)와 해양을 구별한 후 시각화하기
disc = -7     # 기준값
image_mask = image1127.gt(disc)       #changing your image variable name

# Import the Folium library.
import folium

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 9)

# Add the layer to the map object.
map.add_ee_layer(image_mask, {'min': [0], 'max': [1]}, 'HH')

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)



*   분석 결과, 주변에 있는 얼음까지 면적을 같이 계산하였기 때문에 그래프 상에서 빙하면적이 증가되어 나타난 것이다





##### 2022년 1월 14일 
> 빙하 면적 0에 수렴하게 감소 현상



In [ ]:
# 2022년 1월 14일 경의 Sentinel-1 이미지 
start = ee.Date('2022-01-13')
end = ee.Date('2022-01-14')

# Sentinel-1 SAR 이미지 가져오기
image0114 = (ee.ImageCollection('COPERNICUS/S1_GRD')     # changing your image variable name
    .filterDate(start, end)
    .filterBounds(roi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
    .select('HH')
    .mean()
    .clip(roi))

# shapefile (.shp)을 이용하여 roi (region of interest) 설정하기
shp = ee.FeatureCollection('projects/ee-jieun/assets/A77') # check
coor = shp.geometry().getInfo()['coordinates']
roi = ee.Geometry.Polygon(coor)

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 8)

# Add the layer to the map object.
map.add_ee_layer(image0114, {'min': [-22], 'max': [2]}, 'HH')    # changing your image variable name

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)

In [ ]:
# 빙하 (glacier)와 해양을 구별한 후 시각화하기
disc = -7  # 기준값
image_mask = image0114.gt(disc)       #changing your image variable name

# Import the Folium library.
import folium

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
map = folium.Map(location=[-70.6108, -9.4135], zoom_start = 9)

# Add the layer to the map object.
map.add_ee_layer(image_mask, {'min': [0], 'max': [1]}, 'HH')

# Add a layer control panel to the map
map.add_child(folium.LayerControl())

# Display the map.
display(map)



*   분석 결과, 기준값(-7)이 빙하를 제대로 구분하지 못하여 빙하 면적을 0으로 계산하였다



#### 4) 이상치 제거 후 시계열 그래프 다시 보기



> 이상치 제거하여 데이터셋 전처리



In [ ]:
# 2021년 11월 27일 증가한 sumArea 데이터와 2022년 1월 감소한 sumArea 데이터 확인
images_df_cln[(images_df_cln['Month']==11) | ((images_df_cln['Year']==2022) & (images_df_cln['Month']==1))]

In [ ]:
# index = 49,50,55 데이터 제거
index = [49,50,55]
images_df_pro = images_df_cln.drop(index, axis=0)
images_df_pro



> **전처리 한 후 최종 그래프**



In [ ]:
# 최종그래프
iceberg_name = 'A77'
ts_pro = pd.DataFrame({'date': images_df_pro['Timestamp'], 'sumArea': images_df_pro['sumArea']})
alt.Chart(ts_pro, width=1000).mark_line().encode(x='date', y='sumArea').properties(title='시간에 따른 '+iceberg_name +' 빙하 면적 그래프')